### Libs

In [6]:
import pandas as pd
import requests
import os
import numpy as np
import json
import ast
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

### API key

In [ ]:
# Personal key
key = ""

### Retry

In [3]:
retry_strategy = Retry(
    total=5,  # Total number of retries
    backoff_factor=1,  # Exponential backoff
    status_forcelist=[429, 500, 502, 503, 504],  # HTTP codes to retry
    allowed_methods=["GET"], 
    raise_on_status=False  # Don't raise on bad status codes
)

adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("http://", adapter)
http.mount("https://", adapter)

### Check available APIs

Also available directly at: https://api.steampowered.com/ISteamWebAPIUtil/GetSupportedAPIList/v0001/

If we add the key parameters it shows us the APIs only available using the key

In [51]:
pd.set_option('display.max_colwidth', None)

In [62]:
url = f"http://api.steampowered.com/ISteamWebAPIUtil/GetSupportedAPIList/v0001/?key={key}"
response = requests.get(url)
#response.json()['apilist']['interfaces'][0:2]
df = pd.DataFrame(response.json()['apilist']['interfaces'])
#df[df['name']=='ISteamUser']
df[df['name']=='IPlayerService'] # search GetOwnedGames
#df[df['name']=='IEconItems_1046930'] # GetPlayerItems



,name,methods
42,IPlayerService,"[{'name': 'IsPlayingSharedGame', 'version': 1, 'httpmethod': 'GET', 'description': 'Obsolete, partners should use ISteamUser.CheckAppOwnership', 'parameters': [{'name': 'key', 'type': 'string', 'optional': False, 'description': 'Access key'}, {'name': 'steamid', 'type': 'uint64', 'optional': False, 'description': ""The player we're asking about""}, {'name': 'appid_playing', 'type': 'uint32', 'optional': False, 'description': 'The game player is currently playing'}]}, {'name': 'RecordOfflinePlaytime', 'version': 1, 'httpmethod': 'POST', 'description': 'Tracks playtime for a user when they are offline', 'parameters': [{'name': 'steamid', 'type': 'uint64', 'optional': False}, {'name': 'ticket', 'type': 'string', 'optional': False}, {'name': 'play_sessions', 'type': '{message}', 'optional': False}]}, {'name': 'GetRecentlyPlayedGames', 'version': 1, 'httpmethod': 'GET', 'description': 'Gets information about a player's recently played games', 'parameters': [{'name': 'key', 'type': 'string', 'optional': False, 'description': 'Access key'}, {'name': 'steamid', 'type': 'uint64', 'optional': False, 'description': ""The player we're asking about""}, {'name': 'count', 'type': 'uint32', 'optional': False, 'description': 'The number of games to return (0/unset: all)'}]}, {'name': 'GetOwnedGames', 'version': 1, 'httpmethod': 'GET', 'description': 'Return a list of games owned by the player', 'parameters': [{'name': 'key', 'type': 'string', 'optional': False, 'description': 'Access key'}, {'name': 'steamid', 'type': 'uint64', 'optional': False, 'description': ""The player we're asking about""}, {'name': 'include_appinfo', 'type': 'bool', 'optional': False, 'description': 'true if we want additional details (name, icon) about each game'}, {'name': 'include_played_free_games', 'type': 'bool', 'optional': False, 'description': 'Free games are excluded by default. If this is set, free games the user has played will be returned.'}, {'name': 'appids_filter', 'type': 'uint32', 'optional': False, 'description': 'if set, restricts result set to the passed in apps'}, {'name': 'include_free_sub', 'type': 'bool', 'optional': False, 'description': 'Some games are in the free sub, which are excluded by default.'}, {'name': 'skip_unvetted_apps', 'type': 'bool', 'optional': True, 'description': 'if set, skip unvetted store apps'}, {'name': 'language', 'type': 'string', 'optional': False, 'description': 'Will return appinfo in this language'}, {'name': 'include_extended_appinfo', 'type': 'bool', 'optional': False, 'description': 'true if we want even more details (capsule, sortas, and capabilities) about each game. include_appinfo must also be true.'}]}, {'name': 'GetSteamLevel', 'version': 1, 'httpmethod': 'GET', 'description': 'Returns the Steam Level of a user', 'parameters': [{'name': 'key', 'type': 'string', 'optional': False, 'description': 'Access key'}, {'name': 'steamid', 'type': 'uint64', 'optional': False, 'description': ""The player we're asking about""}]}, {'name': 'GetBadges', 'version': 1, 'httpmethod': 'GET', 'description': 'Gets badges that are owned by a specific user', 'parameters': [{'name': 'key', 'type': 'string', 'optional': False, 'description': 'Access key'}, {'name': 'steamid', 'type': 'uint64', 'optional': False, 'description': ""The player we're asking about""}]}, {'name': 'GetCommunityBadgeProgress', 'version': 1, 'httpmethod': 'GET', 'description': 'Gets all the quests needed to get the specified badge, and which are completed', 'parameters': [{'name': 'key', 'type': 'string', 'optional': False, 'description': 'Access key'}, {'name': 'steamid', 'type': 'uint64', 'optional': False, 'description': ""The player we're asking about""}, {'name': 'badgeid', 'type': 'int32', 'optional': False, 'description': ""The badge we're asking about""}]}]"


In [63]:
for api in response.json()['apilist']['interfaces']:
  print(f"{api['name']}\n  - methods:")
  for method in api['methods']:
    print(f"    - {method}")

IClientStats_1046930
  - methods:
    - {'name': 'ReportEvent', 'version': 1, 'httpmethod': 'POST', 'parameters': []}
ICSGOPlayers_730
  - methods:
    - {'name': 'GetNextMatchSharingCode', 'version': 1, 'httpmethod': 'GET', 'parameters': [{'name': 'steamid', 'type': 'uint64', 'optional': False, 'description': 'The SteamID of the user'}, {'name': 'steamidkey', 'type': 'string', 'optional': False, 'description': 'Authentication obtained from the SteamID'}, {'name': 'knowncode', 'type': 'string', 'optional': False, 'description': 'Previously known match sharing code obtained from the SteamID'}]}
ICSGOServers_730
  - methods:
    - {'name': 'GetGameMapsPlaytime', 'version': 1, 'httpmethod': 'GET', 'parameters': [{'name': 'interval', 'type': 'string', 'optional': False, 'description': 'What recent interval is requested, possible values: day, week, month'}, {'name': 'gamemode', 'type': 'string', 'optional': False, 'description': 'What game mode is requested, possible values: competitive, ca

### Explore APIs with games information

In [7]:
# ISteamApps.GetAppList
# All games (and other content) on Steam, with appid and name
# Useful for building a game index.

url = f"http://api.steampowered.com/ISteamApps/GetAppList/v0001/?key={key}&format=json"
response = requests.get(url)
df_games = pd.DataFrame(response.json()['applist']['apps']['app'])
df_games.head()

,appid,name
0,5,Dedicated Server
1,7,Steam Client
2,8,winui2
3,10,Counter-Strike
4,20,Team Fortress Classic


In [8]:
df_games[df_games['name']=='The Elder Scrolls V: Skyrim']

,appid,name
4202,72850,The Elder Scrolls V: Skyrim


In [9]:
# GetNewsForApp
# News from apps (games)
# Could be used for NLP/sentiment trends before game launches or major updates.

# Example: appid=72850 (The Elder Scrolls V: Skyrim)
url = "http://api.steampowered.com/ISteamNews/GetNewsForApp/v0002/?appid=72850&format=json"
response = requests.get(url)
response.json()['appnews']['newsitems'][0]

{'gid': '1802893906766998',
 'title': 'Skyblivion devs used their Bethesda visit to fight the good fight, asking Todd and co to bring back Skyrim-style official modding tutorials',
 'url': 'https://steamstore-a.akamaihd.net/news/externalpost/Rock, Paper, Shotgun/1802893906766998',
 'is_external_url': True,
 'author': '',
 'contents': '<img src="https://assetsio.gnwcdn.com/skyblivion-devs-tell-bethesda-to-bring-modding-tutorials-back-01.jpg?width=1920&height=1920&fit=bounds&quality=70&format=jpg&auto=webp" /> <p>While the prospect <a href="https://www.rockpapershotgun.com/games/the-elder-scrolls-4-oblivion-remake">The Elder Scrolls IV: Oblivion Remastered</a> turning out to be a real thing had folks wondering what that might mean for Oblivion remake mod <a href="https://www.rockpapershotgun.com/games/skyblivion">Skyblivion</a>\'s <a href="https://www.rockpapershotgun.com/skyblivion-project-lead-and-gog-chat-about-partnering-up-to-release-the-massive-elder-scrolls-mod-following-fallout-l

In [10]:
# ISteamUserStats.GetNumberOfCurrentPlayers
# Real-time player count for a given appid
# Useful for game popularity.

#appids = list(range(0,150))
#for appid in appids:
appid = 72850
url = f"http://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v0001/?appid={appid}&format=json"
response = requests.get(url)
response.json()

{'response': {'player_count': 1639, 'result': 1}}

In [19]:
# ISteamUserStats.GetGlobalAchievementPercentagesForApp
# Percent of players who unlocked each achievement.
# Could indicate engagement, difficulty, or completion rate.
#
#appids = list(range(0,1000))
# for appid in appids:
url = f"http://api.steampowered.com/ISteamUserStats/GetGlobalAchievementPercentagesForApp/v0001/?gameid={appid}&format=json"
response = requests.get(url)
response.json()['achievementpercentages']['achievements']['achievement'][:5]

[{'name': 'NEW_ACHIEVEMENT_1_0', 'percent': '88.6'},
 {'name': 'NEW_ACHIEVEMENT_29_0', 'percent': '79.1'},
 {'name': 'NEW_ACHIEVEMENT_47_0', 'percent': '77.2'},
 {'name': 'NEW_ACHIEVEMENT_43_0', 'percent': '75.7'},
 {'name': 'NEW_ACHIEVEMENT_2_0', 'percent': '75.0'}]

### Explore APIs with user information

In [ ]:
# ISteamUser.GetPlayerSummaries
# Profile info like persona name, profile visibility, avatar.

url = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={key}&steamids=76561198080989870&format=json"
response = requests.get(url)
response.json()

{'response': {'players': [{'steamid': '76561198080989870',
    'communityvisibilitystate': 3,
    'profilestate': 1,
    'personaname': 'marra',
    'profileurl': 'https://steamcommunity.com/profiles/76561198080989870/',
    'avatar': 'https://avatars.steamstatic.com/6bedb40cbdedf0d6cea878baf7f4b5bf46e7cf8f.jpg',
    'avatarmedium': 'https://avatars.steamstatic.com/6bedb40cbdedf0d6cea878baf7f4b5bf46e7cf8f_medium.jpg',
    'avatarfull': 'https://avatars.steamstatic.com/6bedb40cbdedf0d6cea878baf7f4b5bf46e7cf8f_full.jpg',
    'avatarhash': '6bedb40cbdedf0d6cea878baf7f4b5bf46e7cf8f',
    'lastlogoff': 1751568444,
    'personastate': 1,
    'realname': 'Lucas Marra',
    'primaryclanid': '103582791429521408',
    'timecreated': 1357738431,
    'personastateflags': 0,
    'loccountrycode': 'BR',
    'locstatecode': '27',
    'loccityid': 8872}]}}

In [22]:
# GetFriendList (v0001)
url = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={key}&steamid=76561198080989870&relationship=friend" # Lucas stream profile
response = requests.get(url)
# friends: 76561197991068074, 76561197995269379, 76561198416187271

response.json()

{'friendslist': {'friends': [{'steamid': '76561197991068074',
    'relationship': 'friend',
    'friend_since': 1388369289},
   {'steamid': '76561197995269379',
    'relationship': 'friend',
    'friend_since': 1387856640},
   {'steamid': '76561198416187271',
    'relationship': 'friend',
    'friend_since': 1719427800}]}}

In [23]:
# GetOwnedGames (v0001)
# GetOwnedGames returns a list of games a player owns along with some playtime information, if the profile is publicly visible. Private, friends-only, and other privacy settings are not supported unless you are asking for your own personal details (ie the WebAPI key you are using is linked to the steamid you are requesting).
url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={key}&steamid=76561198080989870&format=json"
response = requests.get(url)
response.json()

{'response': {'game_count': 10,
  'games': [{'appid': 72850,
    'playtime_forever': 3889,
    'playtime_windows_forever': 0,
    'playtime_mac_forever': 0,
    'playtime_linux_forever': 0,
    'playtime_deck_forever': 0,
    'rtime_last_played': 1513439945,
    'playtime_disconnected': 0},
   {'appid': 271590,
    'playtime_forever': 2084,
    'playtime_windows_forever': 2084,
    'playtime_mac_forever': 0,
    'playtime_linux_forever': 0,
    'playtime_deck_forever': 0,
    'rtime_last_played': 1741453729,
    'playtime_disconnected': 0},
   {'appid': 3240220,
    'playtime_forever': 0,
    'playtime_windows_forever': 0,
    'playtime_mac_forever': 0,
    'playtime_linux_forever': 0,
    'playtime_deck_forever': 0,
    'rtime_last_played': 0,
    'playtime_disconnected': 0},
   {'appid': 489830,
    'playtime_forever': 977,
    'playtime_windows_forever': 967,
    'playtime_mac_forever': 0,
    'playtime_linux_forever': 0,
    'playtime_deck_forever': 0,
    'rtime_last_played': 1730

In [25]:
# GetUserStatsForGame (v0002)
# Returns a list of achievements for this user by app id

url = f"http://api.steampowered.com/ISteamUserStats/GetUserStatsForGame/v0002/?appid=72850&key={key}&steamid=76561198080989870"
response = requests.get(url)
response.json()

{}

In [27]:
# GetPlayerAchievements (v0001)
# Returns a list of achievements for this user by app id

url = f"http://api.steampowered.com/ISteamUserStats/GetPlayerAchievements/v0001/?appid=72850&key={key}&steamid=76561198080989870"
response = requests.get(url)
response.json()

{'playerstats': {'error': 'Profile is not public', 'success': False}}

In [29]:
# GetRecentlyPlayedGames (v0001)
# GetRecentlyPlayedGames returns a list of games a player has played in the last two weeks, if the profile is publicly visible. Private, friends-only, and other privacy settings are not supported unless you are asking for your own personal details (ie the WebAPI key you are using is linked to the steamid you are requesting).

url = f"http://api.steampowered.com/IEconItems_1046930/GetPlayerItems/v0001/?key={key}&steamid=76561198080989870&format=json"
response = requests.get(url)
response.json()

{'result': {'status': 15, 'statusDetail': 'Permission denied'}}

### Get player count by game (to filter most important games)

In [5]:
# Get games ids (appid) and names (name)
url = f"http://api.steampowered.com/ISteamApps/GetAppList/v0001/?key={key}&format=json"
response = requests.get(url)
games = response.json()['applist']['apps']['app']
df_games = pd.DataFrame(games)

In [ ]:
# Save games.csv file with games name and id
df_games.to_csv('../../data/games.csv', index=False)

In [42]:
games[:5]

[{'appid': 5, 'name': 'Dedicated Server'},
 {'appid': 7, 'name': 'Steam Client'},
 {'appid': 8, 'name': 'winui2'},
 {'appid': 10, 'name': 'Counter-Strike'},
 {'appid': 20, 'name': 'Team Fortress Classic'}]

In [ ]:
list_game_player_cnt = []
loop_cnt=0
#for game in games[:3000]:
for game in games:
    dict_game_player_cnt = {}
    url = f"http://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v0001/?appid={game['appid']}&format=json"
    try:
        response = http.get(url, timeout=10)
        data = response.json()
        dict_game_player_cnt['appid'] = game['appid']
        dict_game_player_cnt['name'] = game['name']
        try: # Looks like some games do no have 'player_count'
            dict_game_player_cnt['player_count'] = data['response']['player_count']
        except:
            dict_game_player_cnt['player_count'] = np.nan
        dict_game_player_cnt['result'] = data['response']['result']
        list_game_player_cnt.append(dict_game_player_cnt)
        loop_cnt += 1
        if loop_cnt % 50000 == 0:
            print(loop_cnt)
    except requests.exceptions.RequestException as e:
        print(f"Error getting game {game['appid']} data: {e}")




50000
100000
150000
200000


In [ ]:
df_game_player_cnt = pd.DataFrame(list_game_player_cnt)
df_game_player_cnt_ranked = df_game_player_cnt.sort_values(by='player_count', ascending=False)
df_game_player_cnt_ranked.to_csv('../../data/game_player_cnt_ranked.csv', index=False)

### Get user profile information

In [12]:
pd.set_option('display.max_colwidth', 300)

In [7]:
df_top_1000_game_reviews = pd.read_csv('../../data/top_1000_game_reviews.gz', compression='gzip')

In [ ]:
df_top_1000_game_reviews.columns

Index(['appid', 'recommendationid', 'author', 'language', 'review',
       'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up',
       'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase',
       'received_for_free', 'written_during_early_access',
       'primarily_steam_deck', 'timestamp_dev_responded',
       'developer_response'],
      dtype='object')

In [ ]:
len(df_top_1000_game_reviews)

100000

In [16]:
# steamid — o ID Steam do usuário
# num_games_owned — a quantidade de jogos na conta do usuário
# num_reviews — a quantidade de análises escritas pelo usuário
# playtime_forever — o tempo de uso total do aplicativo pelo usuário
# playtime_last_two_weeks — tempo de uso do aplicativo pelo usuário nas últimas duas semanas
# playtime_at_review — tempo de uso quando a análise foi escrita
# deck_playtime_at_review — tempo de uso quando a análise foi escrita
# last_played — horário da última vez em que o usuário jogou

In [ ]:
#df_top_1000_game_reviews.head(3)

In [ ]:
df_top_1000_game_reviews['steamid'] = df_top_1000_game_reviews['author'].apply(lambda x: ast.literal_eval(x)['steamid'])
df_top_1000_game_reviews['num_games_owned'] = df_top_1000_game_reviews['author'].apply(lambda x: ast.literal_eval(x)['num_games_owned'])
df_top_1000_game_reviews['num_reviews'] = df_top_1000_game_reviews['author'].apply(lambda x: ast.literal_eval(x)['num_reviews'])
# df_top_1000_game_reviews['playtime_forever'] = df_top_1000_game_reviews['author'].apply(lambda x: ast.literal_eval(x)['playtime_forever'])
# df_top_1000_game_reviews['playtime_last_two_weeks'] = df_top_1000_game_reviews['author'].apply(lambda x: ast.literal_eval(x)['playtime_last_two_weeks'])
# df_top_1000_game_reviews['playtime_at_review'] = df_top_1000_game_reviews['author'].apply(lambda x: ast.literal_eval(x)['playtime_at_review'])
# df_top_1000_game_reviews['last_played'] = df_top_1000_game_reviews['author'].apply(lambda x: ast.literal_eval(x)['last_played'])

In [ ]:
list_top_1000_game_reviews = list(set(df_top_1000_game_reviews['steamid']))

In [ ]:
profiles = [] 
start = 0
chunk_size = 100
for i in range(0, len(list_top_1000_game_reviews), chunk_size):
    #list_top_1000_game_reviews_reduced = list_top_1000_game_reviews[start:i]
    list_top_1000_game_reviews_reduced = list_top_1000_game_reviews[i:i+chunk_size]
    url = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={key}&steamids={list_top_1000_game_reviews_reduced}&format=json" # Profile info like persona name, profile visibility, avatar.
    response = requests.get(url)
    profiles.extend(response.json()['response']['players'])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100


In [67]:
len(profiles)

16179

In [ ]:
len(list_top_1000_game_reviews)

16181

In [94]:
df_users_profile = pd.DataFrame(profiles)

In [ ]:
# Save profiles.csv file with users profile
df_users_profile.to_csv('../../data/users_profile.csv', index=False)

### Get user friend list

In [ ]:
# GetFriendList (v0001)
#x = [76561198126277481,76561198177301349]
#x = ['76561198080989870']
users_friendlist = []
for steamid in list_top_1000_game_reviews:
    dict_users_friendlist = {}
    url = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={key}&steamid={steamid}&relationship=friend"
    response = requests.get(url)
    dict_users_friendlist['user_steamid'] =  steamid
    try: # in case of no friends
        dict_users_friendlist['user_friendlist'] =  response.json()['friendslist']['friends']
        users_friendlist.append(dict_users_friendlist)
    except:
        pass

In [17]:
len(users_friendlist)

10351

In [20]:
df_users_friendlist = pd.DataFrame(users_friendlist)

In [21]:
# Save profiles.csv file with users friendlist
df_users_friendlist.to_csv('../../data/users_friendlist.csv', index=False)

### Get user owned games

{'name': 'key', 'type': 'string', 'optional': False, 'description': 'Access key'}, {'name': 'steamid', 'type': 'uint64', 'optional': False, 'description': "The player we're asking about"}, {'name': 'include_appinfo', 'type': 'bool', 'optional': False, 'description': 'true if we want additional details (name, icon) about each game'}, {'name': 'include_played_free_games', 'type': 'bool', 'optional': False, 'description': 'Free games are excluded by default. If this is set, free games the user has played will be returned.'}, {'name': 'appids_filter', 'type': 'uint32', 'optional': False, 'description': 'if set, restricts result set to the passed in apps'}, {'name': 'include_free_sub', 'type': 'bool', 'optional': False, 'description': 'Some games are in the free sub, which are excluded by default.'}, {'name': 'skip_unvetted_apps', 'type': 'bool', 'optional': True, 'description': 'if set, skip unvetted store apps'}, {'name': 'language', 'type': 'string', 'optional': False, 'description': 'Will return appinfo in this language'}, {'name': 'include_extended_appinfo', 'type': 'bool', 'optional': False, 'description': 'true if we want even more details (capsule, sortas, and capabilities) about each game. include_appinfo must also be true.'}]}, 

In [ ]:
# GetOwnedGames (v0001)
# GetOwnedGames returns a list of games a player owns along with some playtime information, if the profile is publicly visible. Private, friends-only, and other privacy settings are not supported unless you are asking for your own personal details (ie the WebAPI key you are using is linked to the steamid you are requesting).
users_owned_games = []
for steamid in list_top_1000_game_reviews:
    dict_users_owned_games = {}
    url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={key}&steamid={steamid}&include_appinfo=true&include_extended_appinfo=true&format=json"
    response = requests.get(url)
    dict_users_owned_games['user_steamid'] =  steamid
    try: # in case of no owned games
        dict_users_owned_games['user_owned_games'] =  response.json()['response']['games']
        users_owned_games.append(dict_users_owned_games)
    except:
        pass

In [46]:
len(users_owned_games)

4332

In [48]:
df_users_owned_games = pd.DataFrame(users_owned_games)

In [49]:
df_users_owned_games.tail()

,user_steamid,user_owned_games
4327,76561199684778078,"[{'appid': 70, 'name': 'Half-Life', 'playtime_forever': 802, 'img_icon_url': '95be6d131fc61f145797317ca437c9765f24b41c', 'capsule_filename': 'library_600x900.jpg', 'has_workshop': False, 'has_market': False, 'has_dlc': True, 'content_descriptorids': [2, 5]}, {'appid': 4000, 'name': 'Garry's Mod'..."
4328,76561198107006111,"[{'appid': 30, 'name': 'Day of Defeat', 'playtime_forever': 0, 'img_icon_url': 'aadc0ce51ff6ba2042d633f8ec033b0de62091d0', 'capsule_filename': 'library_600x900.jpg', 'has_workshop': False, 'has_market': False, 'has_dlc': False, 'content_descriptorids': [2, 5]}, {'appid': 2100, 'name': 'Dark Mess..."
4329,76561198411669195,"[{'appid': 4000, 'name': 'Garry's Mod', 'playtime_forever': 1301, 'img_icon_url': '4a6f25cfa2426445d0d9d6e233408de4d371ce8b', 'has_community_visible_stats': True, 'capsule_filename': 'library_600x900.jpg', 'has_workshop': True, 'has_market': False, 'has_dlc': True}, {'appid': 17390, 'name': 'Spo..."
4330,76561198070842180,"[{'appid': 10, 'name': 'Counter-Strike', 'playtime_forever': 0, 'img_icon_url': '6b0312cda02f5f777efa2f3318c307ff9acafbb5', 'capsule_filename': 'library_600x900.jpg', 'has_workshop': False, 'has_market': False, 'has_dlc': False, 'content_descriptorids': [2, 5]}, {'appid': 80, 'name': 'Counter-St..."
4331,76561197960613683,"[{'appid': 10, 'name': 'Counter-Strike', 'playtime_forever': 0, 'img_icon_url': '6b0312cda02f5f777efa2f3318c307ff9acafbb5', 'capsule_filename': 'library_600x900.jpg', 'has_workshop': False, 'has_market': False, 'has_dlc': False, 'content_descriptorids': [2, 5]}, {'appid': 20, 'name': 'Team Fortr..."


In [50]:
# Save profiles.csv file with users friendlist
df_users_owned_games.to_csv('../../data/users_owned_games.csv', index=False)

### Get user recent played games

In [ ]:
# GetRecentlyPlayedGames (v0001)
# GetRecentlyPlayedGames returns a list of games a player has played in the last two weeks, if the profile is publicly visible. Private, friends-only, and other privacy settings are not supported unless you are asking for your own personal details (ie the WebAPI key you are using is linked to the steamid you are requesting).

users_recently_played_games = []
for steamid in list_top_1000_game_reviews:
    dict_users_recently_played_games = {}
    url = f"http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key={key}&steamid={steamid}&format=json"
    response = requests.get(url)
    dict_users_recently_played_games['user_steamid'] =  steamid
    try: # in case of no recently played games
       dict_users_recently_played_games['user_recently_played_games'] =  response.json()['response']
       users_recently_played_games.append(dict_users_recently_played_games)
    except:
       pass

In [ ]:
users_recently_played_games[5]

{'user_steamid': '76561198073719579',
 'user_recently_played_games': {'total_count': 17,
  'games': [{'appid': 730,
    'name': 'Counter-Strike 2',
    'playtime_2weeks': 16693,
    'playtime_forever': 1640820,
    'img_icon_url': '8dbc71957312bbd3baea65848b545be9eae2a355'},
   {'appid': 3183280,
    'name': 'Game of Thrones: Kingsroad',
    'playtime_2weeks': 14493,
    'playtime_forever': 122773,
    'img_icon_url': '08cfa201f76fa840836f110908992b5fd386419c'},
   {'appid': 3240220,
    'name': 'Grand Theft Auto V Enhanced',
    'playtime_2weeks': 14493,
    'playtime_forever': 147805,
    'img_icon_url': '8355a7bbdb704f727bfba80ec56bc7228991338e'},
   {'appid': 208730,
    'name': 'Game of Thrones ',
    'playtime_2weeks': 14493,
    'playtime_forever': 341926,
    'img_icon_url': '92b3048df9dea8b739560368f56841eb23ad5a8e'},
   {'appid': 753,
    'name': 'Steam',
    'playtime_2weeks': 14493,
    'playtime_forever': 773252,
    'img_icon_url': '1d0167575d746dadea7706685c0f3c01c8aeb6d

In [28]:
len(users_recently_played_games)

16181

In [32]:
df_users_recently_played_games = pd.DataFrame(users_recently_played_games)

In [34]:
df_users_recently_played_games.tail()

,user_steamid,user_recently_played_games
16176,76561198157454615,"{'total_count': 3, 'games': [{'appid': 2088840, 'name': 'Picayune Dreams', 'playtime_2weeks': 1259, 'playtime_forever': 3034, 'img_icon_url': 'dce0cd6551f5654c11844e99e705d61977d8b870'}, {'appid': 1275350, 'name': 'Ninja Kiwi Archive', 'playtime_2weeks': 22, 'playtime_forever': 399, 'img_icon_ur..."
16177,76561198085902463,{}
16178,76561198236398530,{}
16179,76561198003248938,{}
16180,76561198296103889,{}


In [35]:
# Save profiles.csv file with users recently played games
df_users_recently_played_games.to_csv('../../data/users_recently_played_games.csv', index=False)

### Combine user dfs

In [ ]:
df_users_profile = pd.read_csv('../../data/users_profile.csv')
df_users_friendlist = pd.read_csv('../../data/users_friendlist.csv')
df_users_owned_games = pd.read_csv('../../data/users_owned_games.csv')
df_users_recently_played_games = pd.read_csv('../../data/users_recently_played_games.csv')

In [12]:
df_users_recently_played_games.head()

,user_steamid,user_recently_played_games
0,76561198116728538,"{'total_count': 1, 'games': [{'appid': 632360,..."
1,76561198064983001,{}
2,76561198981335772,{}
3,76561198065223916,{}
4,76561198057012202,"{'total_count': 1, 'games': [{'appid': 1681430..."


In [14]:
df_users = df_users_profile.merge(df_users_friendlist, left_on='steamid', right_on='user_steamid', how='left').merge(df_users_owned_games, left_on='steamid', right_on='user_steamid', how='left').merge(df_users_recently_played_games, left_on='steamid', right_on='user_steamid', how='left')

In [17]:
df_users.head()

,steamid,communityvisibilitystate,profilestate,personaname,profileurl,avatar,avatarmedium,avatarfull,avatarhash,personastate,...,gameid,lobbysteamid,gameserverip,gameserversteamid,user_steamid_x,user_friendlist,user_steamid_y,user_owned_games,user_steamid,user_recently_played_games
0,76561198126277481,3,1.0,龍Black Dragon ☯ Syndicate王,https://steamcommunity.com/id/FactaNonVerba/,https://avatars.steamstatic.com/3479e0be80012a...,https://avatars.steamstatic.com/3479e0be80012a...,https://avatars.steamstatic.com/3479e0be80012a...,3479e0be80012aa5fe0e9df66f88ebc86080d10a,0,...,NaN,NaN,NaN,NaN,7.656120e+16,"[{'steamid': '76561197962348788', 'relationshi...",NaN,NaN,76561198126277481,{}
1,76561198177301349,3,1.0,Leo Mystic Magic,https://steamcommunity.com/id/LeoMysticMagic/,https://avatars.steamstatic.com/66221fd9465109...,https://avatars.steamstatic.com/66221fd9465109...,https://avatars.steamstatic.com/66221fd9465109...,66221fd9465109f0a5162804a8de260630401d29,0,...,NaN,NaN,NaN,NaN,7.656120e+16,"[{'steamid': '76561197961632157', 'relationshi...",NaN,NaN,76561198177301349,{}
2,76561198067594892,3,1.0,Khaios,https://steamcommunity.com/id/builds_by_khaios/,https://avatars.steamstatic.com/deb35641a3076b...,https://avatars.steamstatic.com/deb35641a3076b...,https://avatars.steamstatic.com/deb35641a3076b...,deb35641a3076b458f5dd00f1459b844369023e8,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198067594892,{}
3,76561198327562167,3,1.0,st3pis_,https://steamcommunity.com/profiles/7656119832...,https://avatars.steamstatic.com/d248114c7b59d6...,https://avatars.steamstatic.com/d248114c7b59d6...,https://avatars.steamstatic.com/d248114c7b59d6...,d248114c7b59d618653c6e2b62505dbcc9da611c,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198327562167,{}
4,76561198068398113,3,1.0,anarchy,https://steamcommunity.com/id/anarchy753/,https://avatars.steamstatic.com/33480dc39fa29b...,https://avatars.steamstatic.com/33480dc39fa29b...,https://avatars.steamstatic.com/33480dc39fa29b...,33480dc39fa29b328efc5cfada1f35c41cb0def5,0,...,NaN,NaN,NaN,NaN,7.656120e+16,"[{'steamid': '76561197966178706', 'relationshi...",NaN,NaN,76561198068398113,"{'total_count': 3, 'games': [{'appid': 1569580..."
